In [3]:
import arcpy;
import os,sys;
import requests,json;
import importlib;

print("Step 80: Add Station Temperature Data");

import swc_resources;
importlib.reload(swc_resources);
rez = swc_resources.rez();

start_date = '1990-01-01';
end_date   = '2020-12-31';

stations = rez['results']  + os.sep + 'SWC_Station_Universe';
stations_cnt = arcpy.GetCount_management(stations)[0];
print("  Initial SWC Station Universe Count: " + str(stations_cnt));


Step 80: Add Station Temperature Data
  Initial SWC Station Universe Count: 5144


### 80.010: Create fresh SWC_Station_TempMinMax table

In [5]:
%%time

output80010 = rez['results'] + os.sep + 'SWC_Station_TempMinMax';

print("  Creating new station tempminmax table");
if arcpy.Exists(output80010):
    arcpy.Delete_management(output80010);
  
arcpy.CreateTable_management(
     out_path      = os.path.dirname(output80010)
    ,out_name      = os.path.basename(output80010)
);

arcpy.management.AddFields(
     in_table          = output80010
    ,field_description = [
         ['StationId'    ,'TEXT'  ,'Station ID'    ,14 ]
        ,['StationSource','TEXT'  ,'Station Source',255]
        ,['Year'         ,'TEXT'  ,'Year'          ,4  ]
        ,['Month'        ,'TEXT'  ,'Month'         ,2  ]
        ,['Day'          ,'TEXT'  ,'Day'           ,2  ]
        ,['TMax'         ,'DOUBLE','Maximum Temperature']
        ,['TMin'         ,'DOUBLE','Minimum Temperature']
        ,['TSource'      ,'TEXT'  ,'Temperature Source',255]
    ]
);

print("  Adding indexes");
z = arcpy.management.AddIndex(
     in_table   = output80010
    ,fields     = 'StationId'
    ,index_name = 'StationId_IDX'
);

z = arcpy.management.AddIndex(
     in_table   = output80010
    ,fields     = 'StationSource'
    ,index_name = 'StationSource_IDX'
);

z = arcpy.management.AddIndex(
     in_table   = output80010
    ,fields     = 'Year'
    ,index_name = 'Year_IDX'
);

z = arcpy.management.AddIndex(
     in_table   = output80010
    ,fields     = 'Month'
    ,index_name = 'Month_IDX'
);

z = arcpy.management.AddIndex(
     in_table   = output80010
    ,fields     = 'Day'
    ,index_name = 'Day_IDX'
);

print("  Complete.");
del output80010;


  Creating new station tempminmax table
  Adding indexes
  Complete.
Wall time: 6.6 s


### 80.020: Create fresh SWC_Station_TempStats working table

In [6]:
%%time

output80020 = rez['results'] + os.sep + 'SWC_Station_TempStats';

print("  Creating new station tempstat working table");
if arcpy.Exists(output80020):
    arcpy.Delete_management(output80020);
  
arcpy.CreateTable_management(
     out_path      = os.path.dirname(output80020)
    ,out_name      = os.path.basename(output80020)
);

arcpy.management.AddFields(
     in_table          = output80020
    ,field_description = [
         ['StationId'    ,'TEXT'  ,'Station ID'    ,14 ]
        ,['StationSource','TEXT'  ,'Station Source',255]
        ,['Year'         ,'TEXT'  ,'Year'          ,4  ]
        ,['DayCount'     ,'LONG'  ,'Day Count'         ]
    ]
);

print("  Adding indexes");
z = arcpy.management.AddIndex(
     in_table   = output80020
    ,fields     = 'StationId'
    ,index_name = 'StationId_IDX'
);

z = arcpy.management.AddIndex(
     in_table   = output80020
    ,fields     = 'StationSource'
    ,index_name = 'StationSource_IDX'
);

z = arcpy.management.AddIndex(
     in_table   = output80020
    ,fields     = 'Year'
    ,index_name = 'Year_IDX'
);

print("  Complete.");
del output80020;


  Creating new station tempstat working table
  Adding indexes
  Complete.
Wall time: 4.06 s


### 80.020: Loop through each station to add min and max values

In [ ]:
input80020   = rez['resource']  + os.sep + 'SWC_Station_Universe';
output80020  = rez['results']   + os.sep + 'SWC_Station_TempMinMax';
outstat80020 = rez['results']   + os.sep + 'SWC_Station_TempStats';

fldsin = [
     'StationId'
    ,'StationSource'
]

fldsout = [
     'StationId'
    ,'StationSource'
    ,'Year'
    ,'Month'
    ,'Day'
    ,'TMax'
    ,'TMin'
    ,'TSource'
];

fldsstat = [
     'StationId'
    ,'StationSource'
    ,'Year'
    ,'DayCount'
];

with arcpy.da.SearchCursor(
     in_table    = input80020
    ,field_names = fldsin
) as incur:
    
    print("  opening an editor session to support multiple cursors");
    with arcpy.da.Editor(rez['results']) as edit:
    
    outcur = arcpy.da.InsertCursor(
         in_table    = output80020
        ,field_names = fldsout
    );
    
    statcur = arcpy.da.InsertCursor(
         in_table    = outstat80020
        ,field_names = fldsstat
    );
    
    for row in incur:
        
        stationID = row[0];
        stationSource = row[1];
        
        if stationSource == "COOP":
            
            payload = {
                 'dataset'          : 'daily-summaries'
                ,'dataTypes'        : 'TMAX,TMIN'
                ,'stations'         : stationID
                ,'startDate'        : start_date
                ,'endDate'          : end_date
                ,'includeAttributes': 'false'
                ,'format'           : 'json'
                ,'units'            : 'standard'
            }
            r = requests.get(
                'https://www.ncei.noaa.gov/access/services/data/v1'
                ,params = payload        
            );
            rj = r.json();
            
            day_count = 0;
            for reading in rj:
                
                if 'DATE' in reading and 'TMAX' in reading and 'TMIN' in reading:
                    (year,month,day) = reading['DATE'].split('-');
                    
                    
            
                    day_count += 1;
                
        elif stationSource == "ISD":
            pass;
        